In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
sys.path.append("../misc")
import pandas as pd
from misc.MoviaBusDataset import MoviaBusDataset
from misc.data_loader import load_network, adjacency_matrix
import pickle
import os
import yaml

In [3]:
#df = pd.read_hdf("../DCRNN/data/df_highway_2012_4mon_sample.h5")
base_dir = '../DCRNN/movia'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
if not os.path.exists(os.path.join(base_dir,'out')):
    os.makedirs(os.path.join(base_dir,'out'))
if not os.path.exists(os.path.join(base_dir,'data')):
    os.makedirs(os.path.join(base_dir,'data'))
if not os.path.exists(os.path.join(base_dir,'model')):
    os.makedirs(os.path.join(base_dir,'model'))

In [4]:

movia_dataset =  MoviaBusDataset('../data/*/', interpolation=True,agg_time=10)
#movia_dataset.remove_trend()
all_data = pd.concat(movia_dataset.dataframes)
#all_data = all_data.drop(['TimeOfDay'],axis=1)
all_data[all_data.columns] = all_data[all_data.columns].astype('float32')
data_file = os.path.join(base_dir,'movia.h5')
all_data.to_hdf(data_file,key='df',format='table')


In [ ]:
#road_network = load_network()
adj_mx = movia_dataset.adjacency_matrix
adj_mx = adj_mx.astype('float32')
with open(os.path.join(base_dir,'adj_mx.pkl'), 'wb') as f:
    pickle.dump([-1, -1, adj_mx], f, protocol=2)

In [ ]:
%run ../DCRNN/scripts/generate_training_data.py --output_dir ../DCRNN/movia/data --traffic_df_filename ../DCRNN/movia/movia.h5

Generating training data
x shape:  (3829, 6, 192, 2) , y shape:  (3829, 6, 192, 2)
train x:  (2680, 6, 192, 2) y: (2680, 6, 192, 2)
val x:  (383, 6, 192, 2) y: (383, 6, 192, 2)
test x:  (766, 6, 192, 2) y: (766, 6, 192, 2)


In [ ]:
config = {
    'base_dir': 'data/model',
    'log_level': 'INFO',
    'data': {
      'batch_size': 64,
      'dataset_dir': 'movia/data',
      'test_batch_size': 64,
      'val_batch_size': 64,
      'graph_pkl_filename': 'movia/adj_mx.pkl'
    },
    'model': {
      'cl_decay_steps': 2000,
      'filter_type': 'dual_random_walk',
      'horizon': 6,
      'input_dim': 2,
      'l1_decay': 0,
      'max_diffusion_step': 1,
      'num_nodes': len(all_data.columns),
      'num_rnn_layers': 2,
      'output_dim': 1,
      'rnn_units': 64,
      'seq_len': 6,
      'use_curriculum_learning': True
    },
    'train': {
      'base_lr': 0.01,
      'dropout': 0,
      'epoch': 0,
      'epochs': 100,
      'epsilon': 1.0e-3,
      'global_step': 0,
      'lr_decay_ratio': 0.1,
      'max_grad_norm': 5,
      'max_to_keep': 100,
      'min_learning_rate': 2.0e-06,
      'optimizer': 'adam',
      'patience': 50,
      'steps': [20, 30, 40, 50],
      'test_every_n_epochs': 10
    }
}

with open(os.path.join(base_dir,'config.yml'), 'w') as outfile:
    yaml.dump(config, outfile, default_flow_style=False)

In [ ]:
#IF running into ValueError: Variable rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at
#Try restarting kernel
os.chdir(os.path.join(base_dir,'..'))
#%run run_demo.py --config_filename movia/config.yml --output_filename movia/out/out.npz
import tensorflow as tf
import numpy as np

from lib.utils import load_graph_data
from model.dcrnn_supervisor import DCRNNSupervisor
use_cpu_only = False
graph_pkl_filename = 'movia/adj_mx.pkl'
tf_config = tf.ConfigProto()

if use_cpu_only:
    tf_config = tf.ConfigProto(device_count={'GPU': 0})
tf_config.gpu_options.allow_growth = True
_, _, adj_mx = load_graph_data(graph_pkl_filename)
with tf.Session(config=tf_config) as sess:
    supervisor = DCRNNSupervisor(adj_mx=adj_mx, **config)
    supervisor.train(sess)
    #supervisor.load(sess, config['train']['model_filename'])
    outputs = supervisor.evaluate(sess)
    np.savez_compressed(os.path.join(base_dir,'out.npz'), **outputs)
    print('Predictions saved as {}.'.format(os.path.join(base_dir,'out.npz')))

2018-12-10 15:03:01,368 - INFO - Log directory: data/model/dcrnn_DR_1_h_6_64-64_lr_0.01_bs_64_1210150301/
2018-12-10 15:03:01,370 - INFO - {'base_dir': 'data/model', 'log_level': 'INFO', 'data': {'batch_size': 64, 'dataset_dir': 'movia/data', 'test_batch_size': 64, 'val_batch_size': 64, 'graph_pkl_filename': 'movia/adj_mx.pkl'}, 'model': {'cl_decay_steps': 2000, 'filter_type': 'dual_random_walk', 'horizon': 6, 'input_dim': 2, 'l1_decay': 0, 'max_diffusion_step': 1, 'num_nodes': 192, 'num_rnn_layers': 2, 'output_dim': 1, 'rnn_units': 64, 'seq_len': 6, 'use_curriculum_learning': True}, 'train': {'base_lr': 0.01, 'dropout': 0, 'epoch': 0, 'epochs': 100, 'epsilon': 0.001, 'global_step': 0, 'lr_decay_ratio': 0.1, 'max_grad_norm': 5, 'max_to_keep': 100, 'min_learning_rate': 2e-06, 'optimizer': 'adam', 'patience': 50, 'steps': [20, 30, 40, 50], 'test_every_n_epochs': 10}}
2018-12-10 15:03:01,801 - INFO - ('x_train', (2680, 6, 192, 2))
2018-12-10 15:03:01,802 - INFO - ('y_train', (2680, 6, 192

/zhome/30/0/70339/devel/busmodders/DCRNN/lib/utils.py:104: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(d, -1).flatten()


2018-12-10 15:03:06,424 - INFO - Total number of trainable parameters: 223744
2018-12-10 15:03:06,896 - INFO - Start training ...
2018-12-10 15:03:14,165 - INFO - Epoch [0/100] (42) train_mae: 1.6469, val_mae: 1.6318 lr:0.010000 7.2s
2018-12-10 15:03:14,281 - INFO - Val loss decrease from inf to 1.6318, saving to data/model/dcrnn_DR_1_h_6_64-64_lr_0.01_bs_64_1210150301/models-1.6318-42
2018-12-10 15:03:18,304 - INFO - Epoch [1/100] (84) train_mae: 1.5075, val_mae: 1.6189 lr:0.010000 4.0s
2018-12-10 15:03:18,353 - INFO - Val loss decrease from 1.6318 to 1.6189, saving to data/model/dcrnn_DR_1_h_6_64-64_lr_0.01_bs_64_1210150301/models-1.6189-84
2018-12-10 15:03:22,425 - INFO - Epoch [2/100] (126) train_mae: 1.5026, val_mae: 1.6158 lr:0.010000 4.1s
2018-12-10 15:03:22,473 - INFO - Val loss decrease from 1.6189 to 1.6158, saving to data/model/dcrnn_DR_1_h_6_64-64_lr_0.01_bs_64_1210150301/models-1.6158-126
2018-12-10 15:03:26,506 - INFO - Epoch [3/100] (168) train_mae: 1.4991, val_mae: 1.61

2018-12-10 15:05:11,567 - INFO - Epoch [28/100] (1218) train_mae: 1.4640, val_mae: 1.5645 lr:0.001000 4.2s
2018-12-10 15:05:11,617 - INFO - Val loss decrease from 1.5653 to 1.5645, saving to data/model/dcrnn_DR_1_h_6_64-64_lr_0.01_bs_64_1210150301/models-1.5645-1218
2018-12-10 15:05:15,705 - INFO - Epoch [29/100] (1260) train_mae: 1.4634, val_mae: 1.5637 lr:0.001000 4.1s
2018-12-10 15:05:16,472 - INFO - Horizon 01, MAE: 1.43, MAPE: 120197155335.0545, RMSE: 2.01
2018-12-10 15:05:16,478 - INFO - Horizon 02, MAE: 1.46, MAPE: 126242753572.9234, RMSE: 2.05
2018-12-10 15:05:16,483 - INFO - Horizon 03, MAE: 1.47, MAPE: 130279842355.6361, RMSE: 2.07
2018-12-10 15:05:16,489 - INFO - Horizon 04, MAE: 1.48, MAPE: 125048833440.5862, RMSE: 2.09
2018-12-10 15:05:16,495 - INFO - Horizon 05, MAE: 1.50, MAPE: 114675323090.7209, RMSE: 2.11
2018-12-10 15:05:16,501 - INFO - Horizon 06, MAE: 1.51, MAPE: 110600226393.9602, RMSE: 2.13
2018-12-10 15:05:16,548 - INFO - Val loss decrease from 1.5645 to 1.5637, 

2018-12-10 15:07:56,633 - INFO - Epoch [66/100] (2814) train_mae: 1.4624, val_mae: 1.5626 lr:0.000002 4.3s
2018-12-10 15:08:00,891 - INFO - Epoch [67/100] (2856) train_mae: 1.4622, val_mae: 1.5626 lr:0.000002 4.3s
2018-12-10 15:08:05,203 - INFO - Epoch [68/100] (2898) train_mae: 1.4621, val_mae: 1.5626 lr:0.000002 4.3s
2018-12-10 15:08:09,569 - INFO - Epoch [69/100] (2940) train_mae: 1.4621, val_mae: 1.5626 lr:0.000002 4.4s
2018-12-10 15:08:10,404 - INFO - Horizon 01, MAE: 1.43, MAPE: 119924930768.9122, RMSE: 2.01
2018-12-10 15:08:10,410 - INFO - Horizon 02, MAE: 1.46, MAPE: 125912772455.0943, RMSE: 2.05
2018-12-10 15:08:10,416 - INFO - Horizon 03, MAE: 1.47, MAPE: 129668634797.2848, RMSE: 2.07
2018-12-10 15:08:10,422 - INFO - Horizon 04, MAE: 1.48, MAPE: 124146441163.8083, RMSE: 2.09
2018-12-10 15:08:10,428 - INFO - Horizon 05, MAE: 1.49, MAPE: 113867742526.5555, RMSE: 2.11
2018-12-10 15:08:10,434 - INFO - Horizon 06, MAE: 1.51, MAPE: 109748377488.0991, RMSE: 2.13
2018-12-10 15:08:14,

### DIFFUSION = 2
2018-12-10 14:40:25,705 - INFO - Horizon 01, MAE: 1.38, MAPE: 111569731185.0971, RMSE: 1.96 
2018-12-10 14:40:25,712 - INFO - Horizon 02, MAE: 1.39, MAPE: 115525573123.4882, RMSE: 1.98 
2018-12-10 14:40:25,717 - INFO - Horizon 03, MAE: 1.39, MAPE: 118531122300.2530, RMSE: 1.99
2018-12-10 14:40:25,723 - INFO - Horizon 04, MAE: 1.39, MAPE: 107190352152.2177, RMSE: 2.00
2018-12-10 14:40:25,729 - INFO - Horizon 05, MAE: 1.40, MAPE: 103140687758.3068, RMSE: 2.01
2018-12-10 14:40:25,735 - INFO - Horizon 06, MAE: 1.41, MAPE: 102208190672.9279, RMSE: 2.02